In [ ]:
!nvidia-smi

Wed May 10 01:42:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.8 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from tqdm import tqdm

# 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/aiasTerm_Proj/Writing_Style/Writing_Style')

In [ ]:
#파일 불러오기
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test_x.csv")
submission_df = pd.read_csv("sample_submission.csv")

In [ ]:
# Define the function to remove stopwords
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# Define the list of stopwords
stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Preprocess text data
train_text = train_df['text'].values
train_labels = train_df['author'].values
test_text = test_df['text'].values

In [ ]:
# Initialize the ALBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Tokenize the input text data using the ALBERT tokenizer
input_ids = []
attention_masks = []

for text in tqdm(train_text, desc="Preprocessing training data"):
    # Remove stopwords
    text = remove_stopwords(text)
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 512,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(train_labels)

Preprocessing training data: 100%|██████████| 54879/54879 [00:49<00:00, 1115.42it/s]


In [ ]:
# Split the preprocessed training data into training and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = \
    train_test_split(input_ids, train_labels, random_state=42, test_size=0.1, stratify=train_df['author'])
train_masks, validation_masks, _, _ = \
    train_test_split(attention_masks, input_ids, random_state=42, test_size=0.1, stratify=train_df['author'])

In [ ]:
# Set hyperparameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
batch_size = 16
epochs = 3
learning_rate = 5e-5

# Convert inputs, masks, and labels to tensors
train_inputs = torch.tensor(train_inputs)
train_masks = torch.tensor(train_masks)
train_labels = torch.tensor(train_labels)

validation_inputs = torch.tensor(validation_inputs)
validation_masks = torch.tensor(validation_masks)
validation_labels = torch.tensor(validation_labels)

<ipython-input-12-a8755a6bf85a>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
<ipython-input-12-a8755a6bf85a>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_masks = torch.tensor(train_masks)
<ipython-input-12-a8755a6bf85a>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_inputs = torch.tensor(validation_inputs)
<ipython-input-12-a8755a6bf85a>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detac

In [ ]:
# Create a TensorDataset
train_data = TensorDataset(train_inputs, train_masks, train_labels)
# Create a DataLoader for training data
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create a TensorDataset for validation data
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
# Create a DataLoader for validation data
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=5)
model.to(device)

# Set optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
# num_training_steps = epochs * len(train_data) // batch_size
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, total_steps=num_training_steps)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
for epoch in range(1, epochs+1):
    model.train()
    
    train_loss = 0
    for batch in tqdm(train_dataloader):
        input_ids = batch[0].to(device=device)
        attention_masks = batch[1].to(device=device)
        labels = batch[2].to(device=device)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
        
        loss = outputs[0]
        logits = outputs[1]
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        # lr_scheduler.step()

    avg_train_loss = train_loss / len(train_dataloader)
    print(f'Epoch {epoch}/{epochs}, training loss: {avg_train_loss}')

    model.eval()
    
    validation_loss = 0
    for batch in tqdm(validation_dataloader):
        input_ids = batch[0].to(device=device)
        attention_masks = batch[1].to(device=device)
        labels = batch[2].to(device=device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
            loss = outputs[0]
            logits = outputs[1]
            validation_loss += loss.item()
        
    avg_val_loss = validation_loss / len(validation_dataloader)
    val_log_loss = log_loss(labels.cpu(), logits.cpu())
    print(f'Epoch {epoch}/{epochs}, validation loss: {avg_val_loss}, validation log loss: {val_log_loss}')

100%|██████████| 3087/3087 [1:14:16<00:00,  1.44s/it]


Epoch 1/3, training loss: 0.6827376425413492


100%|██████████| 343/343 [03:11<00:00,  1.79it/s]


Epoch 1/3, validation loss: 0.4946118296713245, validation log loss: 0.7673667312851762


100%|██████████| 3087/3087 [1:14:17<00:00,  1.44s/it]


...

Epoch 2/3, validation loss: 0.4263498439798376, validation log loss: 2.755147771161648


100%|██████████| 3087/3087 [1:14:15<00:00,  1.44s/it]


Epoch 3/3, training loss: 0.23077286723856358


100%|██████████| 343/343 [03:10<00:00,  1.80it/s]

Epoch 3/3, validation loss: 0.4491615343931248, validation log loss: 0.5714540124404517


In [ ]:
model.save_pretrained('bert-classifier')

In [ ]:
# Preprocess the test data
test_text = test_df['text'].values

# Tokenize the input text data using the BERT tokenizer
input_ids = []
attention_masks = []

for text in tqdm(test_text, desc="Preprocessing test data"):
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 512,
                        padding='max_length',
                        truncation=True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Set up the data loader for test data
test_data = TensorDataset(input_ids, attention_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

Preprocessing test data: 100%|██████████| 19617/19617 [00:34<00:00, 570.81it/s]


In [ ]:
# Make predictions
model.eval()
test_preds = []

for batch in tqdm(test_dataloader, desc="Making predictions"):
    input_ids = batch[0].to(device)
    attention_masks = batch[1].to(device)

    # Truncate the input sequences that are longer than 512 tokens
    if input_ids.shape[1] > 512:
        input_ids = input_ids[:, :512]
        attention_masks = attention_masks[:, :512]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()

    test_preds.append(logits)

In [ ]:
test_preds = np.concatenate(test_preds, axis=0)

# Create submission file
submission_df[['0', '1', '2', '3', '4']] = test_preds
submission_df.to_csv('BERT-b16-e3-lr5e-5.csv', index=False)